In [17]:
import csv
import pandas as pd
import numpy as np
import math
import sklearn

# Analysis with Ordinal Regression
In this notebook, the parsed data from FRC_Parser was analyzed using ordinal regression.  Before performing ordinal regression, some feature engineering was performed, particularly determining which features were important for each model.  For instance, for the defensive model, the autonomous features should be removed since one is penalized for touching any of the opposing alliance members during autonomous.  In addition, LASSO regularization was performed for variable selection and increased accuracy.  Finally, 

In [13]:
#Read the beta matrices into dataframes
fileLoc = "C:\\Users\\Christian\\Documents\\GitHub\\FRC-Analyzer\\Parsed Output\\combStats_Off.csv"
beta_Off = pd.read_csv(fileLoc,index_col=0)
print(beta_Off.columns.values)
beta_Off.head(10)

fileLoc = "C:\\Users\\Christian\\Documents\\GitHub\\FRC-Analyzer\\Parsed Output\\combStats_Def.csv"
beta_Def = pd.read_csv(fileLoc,index_col=0)
beta_Def.head(10)

['autoRobot.None' 'autoRobot.AutoRun' 'endgameRobot.None'
 'endgameRobot.Parking' 'endgameRobot.Climbing' 'endgameRobot.Levitate'
 'endgameRobot.Unknown' 'adjustPoints' 'autoOwnershipPoints' 'autoPoints'
 'autoQuestRankingPoint' 'autoRunPoints' 'autoScaleOwnershipSec'
 'autoSwitchAtZero' 'autoSwitchOwnershipSec' 'endgamePoints'
 'faceTheBossRankingPoint' 'foulCount' 'foulPoints' 'rp'
 'tba_gameData_LLL' 'tba_gameData_LRL' 'tba_gameData_RLR'
 'tba_gameData_RRR' 'techFoulCount' 'teleopOwnershipPoints' 'teleopPoints'
 'teleopScaleBoostSec' 'teleopScaleForceSec' 'teleopScaleOwnershipSec'
 'teleopSwitchBoostSec' 'teleopSwitchForceSec' 'teleopSwitchOwnershipSec'
 'totalPoints' 'vaultBoostPlayed' 'vaultBoostTotal' 'vaultForcePlayed'
 'vaultForceTotal' 'vaultLevitatePlayed' 'vaultLevitateTotal'
 'vaultPoints' 'rank']


autoRobot.None  autoRobot.AutoRun  endgameRobot.None  \
frc1011               0                  9                  2   
frc1138               1                  8                  1   
frc1165               1                  8                  1   
frc1492               0                  9                  0   
frc1828               6                  3                  8   
frc1836               0                  9                  0   
frc2073               6                  3                  5   
frc2122               0                  9                  0   
frc2135               1                  8                  2   
frc2375               2                  7                  2   

         endgameRobot.Parking  endgameRobot.Climbing  endgameRobot.Levitate  \
frc1011                     3                      1                      3   
frc1138                     4                      3                      1   
frc1165                     7                      0                      1   
frc1492                     7                      0                      2   
frc1828                     0                      0                      1   
frc1836                     2                      2                      5   
frc2073                     2                      0                      2   
frc2122                     5                      0                      4   
frc2135                     6                      1                      0   
frc2375                     4                      0                      3   

         endgameRobot.Unknown  adjustPoints  autoOwnershipPoints  autoPoints  \
frc1011                     0           0.0            -5.988145   -2.374308   
frc1138                     0           0.0             0.436460    3.160595   
frc1165                     0           0.0             6.388329   10.480164   
frc1492                     0           0.0             2.056842    7.083684   
frc1828                     0           0.0            -2.011112    2.479637   
frc1836                     0           0.0            11.344820   15.585831   
frc2073                     0           0.0             2.000373    5.832667   
frc2122                     0           0.0            -0.697102    3.067424   
frc2135                     0           0.0             2.230771    6.941917   
frc2375                     0           0.0            -0.079743    3.580586   

         ...   teleopSwitchOwnershipSec  totalPoints  vaultBoostPlayed  \
frc1011  ...                 -27.794169   -22.561678         -0.662202   
frc1138  ...                  29.131349    81.196106          0.188735   
frc1165  ...                  57.903979   140.047780          0.765376   
frc1492  ...                  44.373917    75.942624          0.199561   
frc1828  ...                  18.152114    42.574413          1.034487   
frc1836  ...                   3.338760    94.438884         -0.025984   
frc2073  ...                  33.571822    82.708019         -0.089901   
frc2122  ...                  16.245898    -5.387790         -0.453328   
frc2135  ...                  56.916077   118.974836          0.637752   
frc2375  ...                  29.589669    65.234041          1.013812   

         vaultBoostTotal  vaultForcePlayed  vaultForceTotal  \
frc1011        -0.381198         -0.236885        -0.373498   
frc1138         0.344464          0.155470         0.049086   
frc1165         0.924305          0.601972         0.767460   
frc1492         0.012820         -0.397862        -0.490965   
frc1828         0.955254          0.293551         0.233334   
frc1836        -0.333536          0.333031         0.339061   
frc2073        -0.031341          0.023829         0.409366   
frc2122        -0.218019          0.132231         0.200603   
frc2135         0.792226          0.720276         0.722077   
frc2375         1.424926         -0.145726        -0.290522   

         vaultLevitatePlayed  vau